In [2]:
%load_ext rpy2.ipython
%matplotlib inline

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import numpy.random as rnd
from scipy import stats
import sympy as sym
from IPython.display import Image

plt.rcParams['figure.figsize'] = (20, 7)


# Rare-event simulation
## Lecture 4
### Patrick Laub, Institut de Science Financière et d’Assurances

### Just need $f_X$ up to a constant

Want to estimate $\ell = \mathbb{P}(X > \gamma)$. To simulate from $(X \mid X > \ell)$ with density

$$ f_{X \mid X > \gamma}(x) = \frac{1\{ x > \gamma \} f_X(x)}{\ell} $$

note that 

$$ \frac{ f_{X \mid X > \gamma}(x) }{ f_{X \mid X > \gamma}(y) } = \frac{ \frac{1\{ x > \gamma \} f_X(x)}{\ell} }{ \frac{1\{ y > \gamma \} f_X(y)}{\ell} } = \frac{ 1\{ x > \gamma \} f_X(x) }{ 1\{ y > \gamma \} f_X(y) }  . $$

## Markov property

__Markov chains taking real values__: Discrete time process $(X_n)_{n \ge 0}$ where $X_n \in \mathbb{R}$), with the _Markov property_:

$$ \mathbb{P}(X_n \le x \mid X_0 = x_0, X_1 = x_1, \dots, X_{n-1} = x_{n-1}) = \mathbb{P}(X_n \le x \mid X_{n-1} = x_{n-1}) $$

for $x \in \mathbb{R}$. 

__Markov chains taking vector values__:

Discrete time process $(\mathbf{X}_n)_{n \ge 0}$, where the process takes values inside a state space $\mathbf{X}_n \in \mathbb{R}^n$, with the _Markov property_:

$$ \mathbb{P}(\mathbf{X}_n \in A \mid \mathbf{X}_0 = \mathbf{x}_0, \mathbf{X}_1 = \mathbf{x}_1, \dots, \mathbf{X}_{n-1} = \mathbf{x}_{n-1}) = \mathbb{P}(\mathbf{X}_n \in A \mid \mathbf{X}_{n-1} = \mathbf{x}_{n-1}) $$

for $A \in \mathcal{B}(\mathbb{R}^n)$. 

__Simplify notation__: Write the chain as $X_1, X_2,\dots$ even if they are vector-valued. Say that they take values inside the _state space_ $\mathcal{S}$ which can either be $\mathbb{R}$ or $\mathbb{R}^n$ or something else.

## Transition kernel

Say $p(y \mid x)$ is the MC's _transition kernel_, which gives the density of proposing a jump to $y$ given we're currently at $x$. Specifically, 

$$ f_{X_{i} \mid X_{i-1}}(y \mid x) = \mathbb{P}(X_i \in [y + \mathrm{d}y] \mid X_{i-1}=x) = p(y \mid x) .$$

So, for every $x$, then $\int_{\mathcal{S}} p(y \mid x) \, \mathrm{d}y = 1$ .

## Stationary distributions

Some MCs have a stationary distribution, $\pi(\,\cdot\,)$, so if

$$ X_0 \sim \pi(\,\cdot\,) \quad\Rightarrow\quad X_1 \sim \pi(\,\cdot\,) \quad\Rightarrow\quad X_2 \sim \pi(\,\cdot\,) \dots $$

To be specific, if we have a MC with transition kernel $p(y \mid x)$ and which takes values in $\mathcal{S}$, 

$$ 
\begin{aligned}
f_{X_1}(x_1) 
&= \int_{x_0 \in \mathcal{S}} f_{X_0,X_1}(x_0, x_1) \, \mathrm{d} x_0
= \int_{x_0 \in \mathcal{S}} f_{X_0}(x_0) f_{X_1 \mid X_0}(x_1 \mid x_0)  \, \mathrm{d} x_0 \\
&= \int_{x_0 \in \mathcal{S}} f_{X_0}(x_0) p(x_1 \mid x_0) \, \mathrm{d} x_0
= \pi(x_1) .
\end{aligned}
$$

If $X_0$ has p.d.f. $f_{X_0}(x_0) = \pi(x_0)$ and 

$$ f_{X_1}(x_1)
= \int_{x_0 \in \mathcal{S}} \pi(x_0) p(x_1 \mid x_0) \, \mathrm{d} x_0
= \pi(x_1)
$$

then $\pi$ is a stationary distribution.

## Limiting distributions

Some MCs have a limiting distribution, that is, for all starting positions $x_0 \in \mathbb{R}$ as $n \to \infty$

$$ (X_n \mid X_0 = x_0) \overset{\mathcal{D}}{\longrightarrow} \, \pi(\,\cdot\,) .$$

Written another way,

$$ \lim_{n \to \infty} f_{X_n \mid X_0}(x \mid x_0) = \pi(x) .$$


The limiting distribution $\pi$ is the stationary distribution (at least, normally).

So... if we want to simulate from $\pi$, just start $X_0$ anywhere and run the MC for _a while_, then after $N \approx \infty$ steps we have

$$ X_{N} \overset{\mathrm{approx.}}{\sim} \pi(\,\cdot\,) \quad\Rightarrow\quad X_{N+1} \overset{\mathrm{approx.}}{\sim} \pi(\,\cdot\,) \quad\Rightarrow\quad X_{N+2} \overset{\mathrm{approx.}}{\sim} \pi(\,\cdot\,) \dots$$

## Why does this work? Why is this $\alpha(X,Y)$ so special?

The secret relates to the fact that $\alpha$ makes the chain _reversible_.

A Markov chain with transition probability $p(y \mid x)$ is _reversible_ with respect to a density $g$ if

$$ g(x) p(y \mid x) = g(y) p(x \mid y) $$

for all $x,y \in \mathcal{S}$. These are called the _detailed balance equations_.

This next part is a bit complicated:
- Not every MC has a stationary distribution
- For MC's which have a stationary distribution, not all are reversible w.r.t. their stationary distribution
- __But__ for MC's which are reversible w.r.t. some distribution $g$, then $g$ must be a stationary distribution for the MC.

## Proof that reversibility implies stationarity


Want to show that if a MC is reversible w.r.t. some distribution $g$, i.e.

$$ g(x) p(y \mid x) = g(y) p(x \mid y)$$

then $g$ must be a stationary distribution for the MC, i.e. $X_0 \sim g \Rightarrow X_1 \sim g$.

__Proof:__

$$
\begin{aligned}
f_{X_1}(x_1)
&= \int_{x_0 \in \mathcal{S}} f_{X_0}(x_0) p(x_1 \mid x_0) \, \mathrm{d} x_0
= \int_{x_0 \in \mathcal{S}} g(x_0) p(x_1 \mid x_0) \, \mathrm{d} x_0 \\
&= \int_{x_0 \in \mathcal{S}} g(x_1) p(x_0 \mid x_1) \, \mathrm{d} x_0
= g(x_1) \int_{x_0 \in \mathcal{S}} p(x_0 \mid x_1) \, \mathrm{d} x_0
= g(x_1) .
\end{aligned}
$$

## In MCMC we want stationarity, so we enforce reversibility w.r.t. target $f_X$

First, what is the transition kernel for the MCMC chain? Is it $q(y \mid x)$? __No__.

For $y\not=x$ it is

$$ p(y \mid x) = q(y \mid x) \alpha(x, y) $$

and if $y = x$ it is

$$ p(y \mid x) = \int_\mathcal{S} q(z \mid x) [1 - \alpha(x, z)] \,\mathrm{d}z $$

However, proposing points using $q(y \mid x)$ __does__ make the chain reversible w.r.t. the target density $f_X$.

## Proof that $q(y \mid x)$ makes the chain reversible w.r.t. the target density $f_X$

Remember, for $x \not= y$ we have $p(y \mid x) = q(y \mid x) \alpha(x, y) $ and

$$ \alpha(x, y) = \min\bigl\{  \frac{ f_X(y) \, q(x \mid y) 
}{ f_X(x) \, q(y \mid x) } , 1 \bigr\} \quad \text{and} \quad \alpha(y, x) = \min\bigl\{  \frac{ f_X(x) \, q(y \mid x) 
}{ f_X(y) \, q(x \mid y) } , 1 \bigr\} . $$

Do we have $ f_X(x) p(y \mid x) = f_X(y) p(x \mid y)$? If $x = y$, then obviously yes. For $x \not=y$

$$ \begin{aligned}
f_X(x) p(y \mid x) 
&= f_X(x) q(y \mid x) \alpha(x, y) \\
&= f_X(x) q(y \mid x) \min\bigl\{  \frac{ f_X(y) \, q(x \mid y) 
}{ f_X(x) \, q(y \mid x) } , 1 \bigr\} \\
&= \min\bigl\{ f_X(y) \, q(x \mid y) 
, f_X(x) q(y \mid x)  \bigr\} \times \frac{f_X(y) \, q(x \mid y) }{f_X(y) \, q(x \mid y) } \\
&= f_X(y) \, q(x \mid y)  \min\bigl\{ 1, \frac{ f_X(x) q(y \mid x) }{f_X(y) \, q(x \mid y) }  \bigr\}  \\
&= f_X(y) \, q(x \mid y) \, \alpha(y, x) \\
&= f_X(y) \, p(x \mid y) .
\end{aligned} $$